# Критерии Стьюдента

In [57]:
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *

Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

Округлите ответ до четырёх знаков после десятичной точки.

In [5]:
import numpy as np

In [9]:
mean_calcium_norm = 9.5
standart_deviation = 0.4
mean_calcium_fact = 9.57 
number_of_w = 160

In [138]:
# scipy.stats.norm.cdf() - нормальная непрерывная случайная величина.
z = (mean_calcium_fact - mean_calcium_norm) / (standart_deviation / np.sqrt(number_of_w))
p = 2*(1-scipy.stats.norm.cdf(abs(z)))
round(p, 4)

0.0269

Имеются данные о стоимости и размерах 53940 бриллиантов:
diamonds.txt

Отделите 25% случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). 

На обучающей выборке настройте две регрессионные модели:
линейную регрессию с помощью LinearRegression без параметров
случайный лес с помощью RandomForestRegressor с random_state=1.

Какая из моделей лучше предсказывает цену бриллиантов? Сделайте предсказания на тестовой выборке, посчитайте модули отклонений предсказаний от истинных цен. Проверьте гипотезу об одинаковом среднем качестве предсказаний, вычислите достигаемый уровень значимости. Отвергается ли гипотеза об одинаковом качестве моделей против двусторонней альтернативы на уровне значимости α=0.05?

In [12]:
import pandas as pd

In [19]:
data = pd.read_csv('diamonds.txt', sep='\t')
display(data.head(3))

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31


In [18]:
from sklearn import model_selection

In [20]:
# разделяем предикторы и вектор ответов
X = data.drop(['price'], axis=1)
y = data.loc[:, data.columns == 'price']

In [30]:
# отделяем в тестовую выборку 25% наблюдений. значение 25% выставляется по умолчаию, 
# но мы его указываем для себя специально
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=1)

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [32]:
# создаем объект класса LinearRegression и тренеруем его на учебной подвыборке 
linear = LinearRegression()
linear.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [117]:
# делаем предсказание цены 
linear_price_prediction = linear.predict(X_test)
linear_price_prediction

array([[ 778.42235621],
       [6930.46053685],
       [2520.12128307],
       ...,
       [5456.49180326],
       [3630.17181904],
       [1203.67470294]])

In [120]:
# посчитайте модули отклонений предсказаний от истинных цен
abs_linear_predictions_deviation = np.abs(linear_price_prediction - y_test)
abs_linear_predictions_deviation.describe()

,price
count,13485.000000
mean,890.376400
std,1161.068461
min,0.035915
25%,208.160385
50%,485.297677
75%,1098.149270
max,18239.846360


In [84]:
# создаем объект класса RandomForestRegressor и тренеруем его на учебной подвыборке 
# при этом конвертируем целевое значение в одномерный массив методом np.ravel.
# аргумент С означает, что элементы массива отсортированы по индексу
random_forest = RandomForestRegressor(random_state=1)
random_forest.fit(X_train, np.ravel(y_train, order='C'))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [85]:
# делаем предсказание цены 
random_forest_price_prediction = random_forest.predict(X_test)
random_forest_price_prediction

array([ 858.55, 7205.6 , 2108.8 , ..., 4711.  , 3435.3 , 1081.9 ])

In [123]:
# посчитайте модули отклонений предсказаний от истинных цен
abs_random_forest_predictions_deviation = np.abs(random_forest_price_prediction[:, np.newaxis] - y_test)
abs_random_forest_predictions_deviation.describe()

,price
count,13485.000000
mean,802.920517
std,1153.425557
min,0.000000
25%,120.925000
50%,320.600000
75%,977.000000
max,12443.600000


наблюдаем различие значений в предсказаниях алгоритмов 

In [124]:
scipy.stats.ttest_rel(abs_linear_predictions_deviation, 
                      abs_random_forest_predictions_deviation)
# p-value стремится к 0. гипотеза об одинаковом качестве моделей 
# против двусторонней альтернативы на уровне значимости α=0.05 отвергается

Ttest_relResult(statistic=array([13.01772978]), pvalue=array([1.65517458e-38]))

посчитайте 95% доверительный интервал для разности средних абсолютных ошибок предсказаний регрессии и случайного леса. Чему равна его ближайшая к нулю граница? Округлите до десятков (поскольку случайный лес может давать немного разные предсказания в зависимости от версий библиотек, мы просим вас так сильно округлить, чтобы полученное значение наверняка совпало с нашим).

In [141]:
# метод tconfint_mean() объекта описательной статистики DescrStatsW()
# возвращает двусторонний доверительный интервал для 
# взвешенного среднего значения данных
borders = DescrStatsW(abs_linear_predictions_deviation 
            - abs_random_forest_predictions_deviation).tconfint_mean()

In [145]:
print round(borders[0][0]), round(borders[1][0])

74.0 101.0
